# US Government Paycheck Protection Program (PPP) Loans over USD 150k and PPP Fraud Examples
## Data Import, Formatting, EDA, and Feature Engineering

## Notebook Overview
This notebook has two sections that each output a data set:
* All PPP Loans Over USD 150k (with data dictionary)
* Examples of PPP Loan Fraud

For the PPP Loans over 150k, the notebook:
* imports this public, US Government data set and data dictionary 
* Format the data, fill in NAs, and generate new features 

The PPP Fraud Examples are: 
* based upon US Government press releases, public court documents, and media coverage 
* include the company and state, information source, and already public names of individuals associated with the fraud
* comprise a small percentage of the total data, as is the case with many fraud data sets 



## Libraries and Custom Functions

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)
pd.set_option('display.max_colwidth', None)

import os 
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings("ignore")

OptionError: 'Pattern matched multiple keys'

In [ ]:
# may need to install this to open Excel files 
!pip install openpyxl
import openpyxl

In [ ]:
# custom function to calculate IRQs 
def calc_iqr_outliers(df, col, multiplier=1.5):
  '''
  inputs:
    df: dataframe containing the data
    col: specific column to analyze
    multiplier: value to use when calculating outliers; defaults to 1.5

  purpose: 
    calculates the IQR and outliers using the provided multiplier

  outputs: 
    text description of the Q1, Q3, upper and lower limits, and outliers
  '''
  # calculate IQR
  iqr = stats.iqr(df[col], interpolation = 'midpoint')
  print(f'{col} IQR:', round(iqr, 2))

  # calculate quantiles:
  q1 = df[col].quantile(0.25)
  q3 = df[col].quantile(0.75)

  # calculate outliers 
  print(f'using {multiplier} to determine outliers\n')
  lower_lim = round(q1 - (iqr * multiplier), 2)
  upper_lim = round(q3 + (iqr * multiplier), 2)
  lower_count = df[df[col] < lower_lim].shape[0]
  upper_count = df[df[col] > upper_lim].shape[0]
  

  print(f'{col} Q1:', round(q1, 2))
  print(f'{col} lower limit:', round(lower_lim, 2))
  print(f'{col} contains {lower_count} records below the lower limit\n')

  print(f'{col} Q3:', round(q3, 2))
  print(f'upper limit:', round(upper_lim, 2))
  print(f'{col} contains {upper_count} records above the upper limit\n')

  print(f'{col} contains {lower_count + upper_count} total outliers')
  print(f'this represents {round((lower_count + upper_count)/df.shape[0],2)} of the data')


In [ ]:
def format_address(df, col):
  '''
  input: 
    df = dataframe
    col = specific column

  apply multiple text formatting functions

  return the column 
  '''
  df[col] = df[col].str.lower()
  df[col] = df[col].str.replace(r'[^\w\s]+', '')
  df[col] = df[col].str.strip()
  df[col] = df[col].str.replace(r'suite', 'ste')

  return df[col]

In [ ]:
def format_name(df, col):
  '''
  inputs:
    df = dataframe
    col = column
  
  applies the formatting functions 

  returns the column 
  '''
  df[col] = df[col].str.replace(r'[^\w\s]+', '')
  df[col] = df[col].str.lower()
  df[col] = df[col].str.strip()

  return df[col]

### Import PPP Data for USD 150k+

In [ ]:
# import data set 
df = pd.read_csv('https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/501af711-1c91-477a-80ce-bf6428eb9253/download/public_150k_plus_220403.csv')
df.shape

In [ ]:
# import data dictionary 
ppp_dict = pd.read_excel('https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/aab8e9f9-36d1-42e1-b3ba-e59c79f1d7f0/download/ppp-data-dictionary.xlsx')
ppp_dict.shape

In [ ]:
ppp_dict

## PPP Loans: Feature Analysis 

In [ ]:
df.info()

### Loan Number
Loan Number (unique identifier)


In [ ]:
df['LoanNumber'].isna().sum()

### Date Approved
Loan Funded Date

In [ ]:
df['DateApproved'].isna().sum()

In [ ]:
df['DateApproved'] = pd.to_datetime(df['DateApproved'])

In [ ]:
print('Earliest Date Approved:', df['DateApproved'].dt.date.min())
print('Most Revent Date Approved:', df['DateApproved'].dt.date.max())


### SBA Office Code
SBA Origination Office Code

In [ ]:
df['SBAOfficeCode'].isna().sum()

In [ ]:
df['SBAOfficeCode'].value_counts().head()

### Processing Method

In [ ]:
df['ProcessingMethod'].isna().sum()

In [ ]:
df['ProcessingMethod'].value_counts()

### Borrower Name

In [ ]:
df['BorrowerName'].isna().sum()

In [ ]:
df['BorrowerName'].fillna('Unknown', inplace=True)
df['BorrowerName'].isna().sum()

In [ ]:
# remove punctuation 
df['BorrowerName'] = format_name(df = df, col = 'BorrowerName')


In [ ]:
df['BorrowerName'].value_counts().head(10)

### Borrower Address

In [ ]:
df['BorrowerAddress'].isna().sum()

In [ ]:
df['BorrowerAddress'].fillna('Unknown', inplace=True)
df['BorrowerAddress'].isna().sum()

In [ ]:
df['BorrowerAddress'] = format_address(df = df, col = 'BorrowerAddress')

In [ ]:
df['BorrowerAddress'].value_counts().head(10)

### Borrower City

In [ ]:
df['BorrowerCity'].isna().sum()

In [ ]:
df['BorrowerCity'].fillna('Unknown', inplace=True)
df['BorrowerCity'].isna().sum()

In [ ]:
df['BorrowerCity'] = df['BorrowerCity'].str.lower()

In [ ]:
df['BorrowerCity'].value_counts().head(10)

### Borrower State

In [ ]:
df['BorrowerState'].isna().sum()

In [ ]:
df['BorrowerState'].fillna('UNK', inplace=True)
df['BorrowerState'].isna().sum()

In [ ]:
df['BorrowerState'].value_counts().head()

### Borrower Zip

In [ ]:
df['BorrowerZip'].isna().sum()

In [ ]:
df['BorrowerZip'].fillna(00000, inplace=True)
df['BorrowerZip'].isna().sum()

In [ ]:
df['BorrowerZip'].value_counts().head()

### Loan Status Date
Loan Status Date is blank when the loan is disbursed but not Paid In Full or Charged Off

In [ ]:
df['LoanStatusDate'].isna().sum()

In [ ]:
# fill NAs with Loan Date Approved
df['LoanStatusDate'].fillna(df['DateApproved'], inplace=True)
df['LoanStatusDate'].isna().sum()

In [ ]:
df['LoanStatusDate'] = pd.to_datetime(df['LoanStatusDate'])

In [ ]:
print('Earliest Loan Status Date:', df['LoanStatusDate'].dt.date.min())
print('Most Recent Loan Status Date:', df['LoanStatusDate'].dt.date.max())

### Loan Status
Loan Status is replaced by 'Exemption 4' when the loan is disbursed but not Paid in Full or Charged Off

In [ ]:
df['LoanStatus'].isna().sum()

In [ ]:
df['LoanStatus'].value_counts().head()

### Term

Loan Maturity in Months

In [ ]:
df['Term'].isna().sum()

In [ ]:
df['Term'].value_counts().head()

In [ ]:
print('Shortest Loan Term:', df['Term'].min())
print('Longest Loan Term:', df['Term'].max())

### SBA Guaranty Percentage

In [ ]:
df['SBAGuarantyPercentage'].isna().sum()

In [ ]:
print('Lowest Guaranty Percentage:', df['SBAGuarantyPercentage'].min())
print('Highest Guaranty Percentage:', df['SBAGuarantyPercentage'].max())

### Initial Approval Amount
Loan Approval Amount (at origination)

In [ ]:
df['InitialApprovalAmount'].isna().sum()

In [ ]:
round(df['InitialApprovalAmount'].describe(), 2)

In [ ]:
calc_iqr_outliers(df=df, col='InitialApprovalAmount', multiplier=1.5)

In [ ]:
plt.figure(figsize = (12,8))

(df['InitialApprovalAmount'] / 1000).plot(kind='hist', 
                                         bins = 50,
                                         log = False,
                                         title = 'Loan Initial Approval Amount (thousands)')

plt.show()

### Current Approval Amount 


In [ ]:
df['CurrentApprovalAmount'].isna().sum()

In [ ]:
round(df['CurrentApprovalAmount'].describe(), 2)

In [ ]:
calc_iqr_outliers(df=df, col='CurrentApprovalAmount', multiplier=1.5)

In [ ]:
print('Count of records where Current Approval Amount is different than Initial Approval Amount:',
  df[df['CurrentApprovalAmount'] != 
     df['InitialApprovalAmount']].shape[0])

### Undisbersed Amount

In [ ]:
df['UndisbursedAmount'].isna().sum()

In [ ]:
df['UndisbursedAmount'].value_counts().head()

In [ ]:
df['UndisbursedAmount'].fillna(0, inplace=True)
df['UndisbursedAmount'].isna().sum()

### Franchise Name

In [ ]:
df['FranchiseName'].isna().sum()

In [ ]:
df['FranchiseName'].fillna('NonFranchise', inplace=True)
df['FranchiseName'].isna().sum()

In [ ]:
df[df['FranchiseName'] != 'NonFranchise']['FranchiseName'].value_counts().head(10)

### Servicing Lender Location ID

Lender Location ID (unique identifier)

In [ ]:
df['ServicingLenderLocationID'].isna().sum()

### Servicing Lender Name

In [ ]:
df['ServicingLenderName'].isna().sum()

In [ ]:
df['ServicingLenderName'] = format_name(df = df, col='ServicingLenderName')

In [ ]:
df['ServicingLenderName'].value_counts().head(10)

### ServicingLenderAddress

In [ ]:
df['ServicingLenderAddress'].isna().sum()

In [ ]:
df['ServicingLenderAddress'] = format_address(df = df, col = 'ServicingLenderAddress')

### Servicing Lender City

In [ ]:
df['ServicingLenderCity'].isna().sum()

In [ ]:
df['ServicingLenderCity'] = df['ServicingLenderCity'].str.lower()

In [ ]:
df['ServicingLenderCity'].value_counts().head()

### Servicing Lender State

In [ ]:
df['ServicingLenderState'].isna().sum()

### Servicing Lender Zip

In [ ]:
df['ServicingLenderZip'].isna().sum()

### Rural Urban Indicator
Rural or Urban Indicator (R/U)

In [ ]:
df['RuralUrbanIndicator'].isnull().sum()

In [ ]:
df['RuralUrbanIndicator'].value_counts()

### Hubzone Indicator


In [ ]:
df['HubzoneIndicator'].isna().sum()

In [ ]:
df['HubzoneIndicator'].value_counts()

### LMI Indicator

In [ ]:
df['LMIIndicator'].isna().sum()

In [ ]:
df['LMIIndicator'].value_counts()

### Business Age Description

In [ ]:
df['BusinessAgeDescription'].isna().sum()

In [ ]:
df['BusinessAgeDescription'].fillna('Unanswered', inplace=True)

In [ ]:
df['BusinessAgeDescription'].value_counts()

### Project City

In [ ]:
df['ProjectCity'].isna().sum()

In [ ]:
df['ProjectCity'].fillna('Unknown', inplace=True)
df['ProjectCity'].isna().sum()

In [ ]:
df['ProjectCity'] = df['ProjectCity'].str.lower()

In [ ]:
df['ProjectCity'].value_counts().head(10)

### Project County Name

In [ ]:
df['ProjectCountyName'].isna().sum()

In [ ]:
df['ProjectCountyName'].fillna('NotProvided', inplace=True)
df['ProjectCountyName'].isna().sum()

In [ ]:
df['ProjectCountyName'] = df['ProjectCountyName'].str.lower()

In [ ]:
df['ProjectCountyName'].value_counts().head()

### Project State

In [ ]:
df['ProjectState'].isna().sum()

In [ ]:
df['ProjectState'].fillna('UNK', inplace=True)
df['ProjectState'].isna().sum()

In [ ]:
df['ProjectState'].value_counts().head()

### Project Zip

In [ ]:
df['ProjectZip'].isna().sum()

In [ ]:
df['ProjectZip'].fillna(00000, inplace=True)
df['ProjectZip'].isna().sum()

In [ ]:
df['ProjectZip'].value_counts().head()

## CD
Project Congressional District 

In [ ]:
df['CD'].isna().sum()

In [ ]:
# fill in the only missing value 
df['CD'].fillna('UNK', inplace=True)
df['CD'].isna().sum()

In [ ]:
df['CD'].value_counts().head()

### Jobs Reported

In [ ]:
df['JobsReported'].isna().sum()

In [ ]:
round(df['JobsReported'].describe(),2)

In [ ]:
calc_iqr_outliers(df=df, col='JobsReported')

In [ ]:
df['JobsReported'].fillna(df['JobsReported'].median(), inplace=True)
df['JobsReported'].isna().sum()

### NAICS Code
North American Industry Classification System (NAICS) Code

In [ ]:
df['NAICSCode'].isna().sum()

In [ ]:
# fill NA with 81, which stands for 'Other Services'
df['NAICSCode'].fillna(81, inplace=True)

In [ ]:
df['NAICSCode'] = df['NAICSCode'].astype(int)
df['NAICSCode'] = df['NAICSCode'].astype('category')

In [ ]:
df['NAICSCode'].value_counts().head()

### Race
Borrower Race Description

In [ ]:
df['Race'].isna().sum()

In [ ]:
df['Race'].value_counts()

### Ethnicity

In [ ]:
df['Ethnicity'].isna().sum()

In [ ]:
df['Ethnicity'].value_counts()

### Note: Proceed data is lender reported at origination. On the PPP application the proceeds fields were check boxes

*Will fill all empty PROCEED features with 0*

### Utilities Proceed

In [ ]:
df['UTILITIES_PROCEED'].isna().sum()

In [ ]:
# already has many zeros so filter for more than that 
round(df[df['UTILITIES_PROCEED'] > 0]['UTILITIES_PROCEED'].describe(), 2)

In [ ]:
df['UTILITIES_PROCEED'].fillna(0, inplace=True)

### Payroll Proceed

In [ ]:
df['PAYROLL_PROCEED'].isna().sum()

In [ ]:
round(df['PAYROLL_PROCEED'].describe(), 2)

In [ ]:
df['PAYROLL_PROCEED'].fillna(0, inplace=True)
df['PAYROLL_PROCEED'].isna().sum()

### Mortgage Interest Proceed

In [ ]:
df['MORTGAGE_INTEREST_PROCEED'].isna().sum()

In [ ]:
round(df['MORTGAGE_INTEREST_PROCEED'].describe(), 2)

In [ ]:
df['MORTGAGE_INTEREST_PROCEED'].fillna(0, inplace=True)
df['MORTGAGE_INTEREST_PROCEED'].isna().sum()

### Rent Proceed

In [ ]:
df['RENT_PROCEED'].isna().sum()

In [ ]:
round(df['RENT_PROCEED'].describe(), 2)

In [ ]:
df['RENT_PROCEED'].fillna(0, inplace=True)
df['RENT_PROCEED'].isna().sum()

### Refinance EIDL Proceed

In [ ]:
df['REFINANCE_EIDL_PROCEED'].isna().sum()

In [ ]:
round(df['REFINANCE_EIDL_PROCEED'].describe(), 2)

In [ ]:
df['REFINANCE_EIDL_PROCEED'].fillna(0, inplace=True)
df['REFINANCE_EIDL_PROCEED'].isna().sum()

### Health Care Proceed

In [ ]:
df['HEALTH_CARE_PROCEED'].isna().sum()

In [ ]:
round(df['HEALTH_CARE_PROCEED'].describe(), 2)

In [ ]:
df['HEALTH_CARE_PROCEED'].fillna(0, inplace=True)
df['HEALTH_CARE_PROCEED'].isna().sum()

### Debt Interest Proceed

In [ ]:
df['DEBT_INTEREST_PROCEED'].isna().sum()

In [ ]:
round(df['DEBT_INTEREST_PROCEED'].describe(), 2)

In [ ]:
df['DEBT_INTEREST_PROCEED'].fillna(0, inplace=True)
df['DEBT_INTEREST_PROCEED'].isna().sum()

### Business Type

In [ ]:
df['BusinessType'].isna().sum()

In [ ]:
df['BusinessType'].fillna('Unknown', inplace=True)
df['BusinessType'].value_counts()

In [ ]:
df['BusinessType'].isna().sum()

### Originating Lender Location ID

In [ ]:
df['OriginatingLenderLocationID'].isna().sum()

### Originating Lender

In [ ]:
df['OriginatingLender'].isna().sum()

In [ ]:
df['OriginatingLender'] = format_name(df = df, col='OriginatingLender')

In [ ]:
df['OriginatingLender'].value_counts().head()

### Originating Lender City

In [ ]:
df['OriginatingLenderCity'].isna().sum()

In [ ]:
df['OriginatingLenderCity'] = df['OriginatingLenderCity'].str.lower()

In [ ]:
df['OriginatingLenderCity'].value_counts().head()

### Originating Lender State

In [ ]:
df['OriginatingLenderState'].isna().sum()

In [ ]:
df['OriginatingLenderState'].value_counts().head()

### Gender


In [ ]:
df['Gender'].isna().sum()

In [ ]:
df['Gender'].value_counts()

### Veteran

In [ ]:
df['Veteran'].isna().sum()

In [ ]:
df['Veteran'].value_counts()

### Nonprofit

In [ ]:
df['NonProfit'].isna().sum()

In [ ]:
df['NonProfit'].fillna('UNK', inplace=True)

In [ ]:
df['NonProfit'].value_counts()

### Forgiveness Amount


In [ ]:
df['ForgivenessAmount'].isna().sum()

In [ ]:
round(df['ForgivenessAmount'].describe(), 2)

In [ ]:
df['ForgivenessAmount'].fillna(0, inplace=True)
df['ForgivenessAmount'].isna().sum()

### Forgiveness Date

In [ ]:
df['ForgivenessDate'].isna().sum()

In [ ]:
df['ForgivenessDate'].fillna('01/01/1900', inplace=True)

In [ ]:
df['ForgivenessDate'] = pd.to_datetime(df['ForgivenessDate'])

## Feature Engineering 

### Create feature of Approval Difference

In [ ]:
df['ApprovalDiff'] = df['CurrentApprovalAmount'] - df['InitialApprovalAmount']

In [ ]:
print('Percentage of loans with changed amount:',
      round(df[df['ApprovalDiff'] != 0].shape[0] / df.shape[0],2))

print('Percentage of loans where amount increased:',
      round(df[df['ApprovalDiff'] > 0].shape[0] / df.shape[0],2))

print('Percentage of loans where amount decreased:',
      round(df[df['ApprovalDiff'] < 0].shape[0] / df.shape[0],2))

In [ ]:
round(df['ApprovalDiff'].describe(),2)

In [ ]:
calc_iqr_outliers(df=df, col='ApprovalDiff', multiplier=1.5)

### NotForgivenAmount
*Feature created to show difference between Updated Loan Amount and Forgiven Amount*

In [ ]:
df['NotForgivenAmount'] = df['CurrentApprovalAmount'] - df['ForgivenessAmount']

In [ ]:
df['NotForgivenAmount'].isna().sum()

In [ ]:
round(df['NotForgivenAmount'].describe())

### Forgiven Percentage
*Feature created to show the percentage of the loan forgiven*

In [ ]:
df['ForgivenPercentage'] = round(df['ForgivenessAmount']/df['CurrentApprovalAmount'], 2)

In [ ]:
df['ForgivenPercentage'].isna().sum()

In [ ]:
round(df['ForgivenPercentage'].describe(), 2)

### TOTAL_PROCEED

* Create a feature adding up all the PROCEED colums
* May help identify anomalies if the total is significantly different from the loan amount

In [ ]:
df['TOTAL_PROCEED'] = (df['UTILITIES_PROCEED'] + 
                       df['PAYROLL_PROCEED'] + 
                       df['MORTGAGE_INTEREST_PROCEED'] + 
                       df['RENT_PROCEED'] + 
                       df['REFINANCE_EIDL_PROCEED'] + 
                       df['HEALTH_CARE_PROCEED'] + 
                       df['DEBT_INTEREST_PROCEED'])

### PROCEED_DIFF

Difference between CurrentApprovalAmount - TOTAL_PROCEED

In [ ]:
df['PROCEED_Diff'] = df['CurrentApprovalAmount'] - df['TOTAL_PROCEED']

### PROCEED Percentages
* Create a new feature from each PROCEED feature with the percentage of total 
* Uses these to identify potential outliers in the data based on claimed loan allocation 

In [ ]:
df['UTILITIES_PROCEED_pct'] = round(df['UTILITIES_PROCEED'] / df['CurrentApprovalAmount'], 3)
df['PAYROLL_PROCEED_pct'] = round(df['PAYROLL_PROCEED'] / df['CurrentApprovalAmount'], 3)
df['MORTGAGE_INTEREST_PROCEED_pct'] = round(df['MORTGAGE_INTEREST_PROCEED'] / df['CurrentApprovalAmount'], 3)
df['RENT_PROCEED_pct'] = round(df['RENT_PROCEED'] / df['CurrentApprovalAmount'], 3)
df['REFINANCE_EIDL_PROCEED_pct'] = round(df['REFINANCE_EIDL_PROCEED'] / df['CurrentApprovalAmount'], 3)
df['HEALTH_CARE_PROCEED_pct'] = round(df['HEALTH_CARE_PROCEED'] / df['CurrentApprovalAmount'], 3)
df['DEBT_INTEREST_PROCEED_pct'] = round(df['DEBT_INTEREST_PROCEED'] / df['CurrentApprovalAmount'], 3)

### PROCEED Per Job

PAYROLL_PROCEED / JobsReported

In [ ]:
df['PROCEED_Per_Job'] = round(df['PAYROLL_PROCEED'] / df['JobsReported'], 2)

In [ ]:
# df.head()

### Update Data Dictionary 

In [ ]:
# df.columns

In [ ]:
ppp_dict

In [ ]:
ppp_dict_v2 = pd.DataFrame(df.columns, columns=['Field Name'])

In [ ]:
ppp_dict_v2 = pd.merge(left = ppp_dict_v2, 
                       right = ppp_dict,
                       how = 'left',
                       on = 'Field Name'
)
ppp_dict_v2.set_index('Field Name', inplace=True)

In [ ]:
ppp_dict_v2.head

In [ ]:
ppp_dict_v2[ppp_dict_v2['Field Description'].isna()]

In [ ]:
ppp_dict_v2.at['UTILITIES_PROCEED', 'Field Description'] = 'Loan proceeds (dollars) to be used for utilities'
ppp_dict_v2.at['PAYROLL_PROCEED', 'Field Description'] = 'Loan proceeds (dollars) to be used for payroll'
ppp_dict_v2.at['MORTGAGE_INTEREST_PROCEED', 'Field Description'] = 'Loan proceeds (dollars) to be used for mortgage interest'
ppp_dict_v2.at['RENT_PROCEED', 'Field Description'] = 'Loan proceeds (dollars) to be used for rent'
ppp_dict_v2.at['REFINANCE_EIDL_PROCEED', 'Field Description'] = 'Loan proceeds (dollars) to be used to refinance an Economic Injury Disaster Loan'
ppp_dict_v2.at['HEALTH_CARE_PROCEED', 'Field Description'] = 'Loan proceeds (dollars) to be used for health care costs'
ppp_dict_v2.at['DEBT_INTEREST_PROCEED', 'Field Description'] = 'Loan proceeds (dollars) to be used for debt interest payments'

ppp_dict_v2.at['ApprovalDiff', 'Field Description'] = 'Difference between CurrentApprovalAmount and InitialApprovalAmount'
ppp_dict_v2.at['NotForgivenAmount', 'Field Description'] = 'Loan amount that has not been forgiven'
ppp_dict_v2.at['ForgivenPercentage', 'Field Description'] = 'Percentage of the CurrentApprovalAmount that has been forgiven'
ppp_dict_v2.at['TOTAL_PROCEED', 'Field Description'] = 'Sum of all prior PROCEED columns'
ppp_dict_v2.at['PROCEED_Diff', 'Field Description'] = 'Difference between CurrentApprovalAmount and TOTAL_PROCEED'

ppp_dict_v2.at['UTILITIES_PROCEED_pct', 'Field Description'] = 'UTILITIES_PROCEED as a percentage of CurrentApprovalAmount'
ppp_dict_v2.at['PAYROLL_PROCEED_pct', 'Field Description'] = 'PAYROLL_PROCEED as a percentage of CurrentApprovalAmount'
ppp_dict_v2.at['MORTGAGE_INTEREST_PROCEED_pct', 'Field Description'] = 'MORTGAGE_INTEREST_PROCEED as a percentage of CurrentApprovalAmount'
ppp_dict_v2.at['RENT_PROCEED_pct', 'Field Description'] = 'RENT_PROCEED as a percentage of CurrentApprovalAmount'
ppp_dict_v2.at['REFINANCE_EIDL_PROCEED_pct', 'Field Description'] = 'REFINANCE_EIDL_PROCEED as a percentage of CurrentApprovalAmount'
ppp_dict_v2.at['HEALTH_CARE_PROCEED_pct', 'Field Description'] = 'HEALTH_CARE_PROCEED as a percentage of CurrentApprovalAmount'
ppp_dict_v2.at['DEBT_INTEREST_PROCEED_pct', 'Field Description'] = 'DEBT_INTEREST_PROCEED as a percentage of CurrentApprovalAmount'

ppp_dict_v2.at['PROCEED_Per_Job', 'Field Description'] = 'PAYROLL_PROCEED divided by JobsReported'


In [ ]:
ppp_dict_v2.reset_index(inplace=True)

In [ ]:
# ppp_dict_v2

## Fraud Examples

* Create a feature labeling potential or actual fraud examples based upon government indictments and prosecutions
  * Fraud: Conviction, guitly plea, or indictment
  * Unknown: No public conviction, guilty plea, or indictment
* Individuals and businesses are innocent until proven guilty
* For this data set, 'Fraud' will include all indictments, convictions, and guilty please due to the low instance of identified fraud and the assumption that cases where the government files charges display risk indicative of fraud 

*An indictment is merely an allegation and all defendants are presumed innocent until proven guilty beyond a reasonable doubt in a court of law.*

In [ ]:
# # query to check names 
# df[df['BorrowerName'].str.contains('', flags=re.IGNORECASE)]

In [ ]:
ppp_fraud_cases = {
               'Apollo HP Inc':  {'State': 'CA', 
                                  'Source': 'https://www.justice.gov/usao-edca/pr/rocklin-business-owner-pleads-guilty-defrauding-banks-while-awaiting-sentencing-other',
                                  'Suspects': [['Kevin Lee Co', 'CA']]
                                  },
               'Power House Sports Academy': {'State': 'GA',
                                              'Source': 'https://www.justice.gov/opa/pr/six-charged-connection-3-million-paycheck-protection-program-fraud-scheme',
                                              'Suspects': [['Rodericque Thompson', 'GA'],
                                                          ['Micah K Baisden', 'GA']]
                                              },
              'Faithful Transport Services LLC': {'State': 'SC',
                                                  'Source': 'https://www.justice.gov/opa/pr/six-charged-connection-3-million-paycheck-protection-program-fraud-scheme',
                                                  'Suspects': [['Rodericque Thompson', 'GA'],
                                                               ['Travis C Crosby', 'SC']]
                                                  },
              'KMJ Transport LLC': {'State': 'SC',
                                    'Source': 'https://www.justice.gov/opa/pr/six-charged-connection-3-million-paycheck-protection-program-fraud-scheme',
                                    'Suspects': [['Rodericque Thompson', 'GA'],
                                                 ['Keith A Maloney Jr', 'GA']]
                                    },
              'Market Yourself Inc': {'State': 'GA',
                                      'Source': 'https://www.justice.gov/opa/pr/six-charged-connection-3-million-paycheck-protection-program-fraud-scheme',
                                      'Suspects': [['Rodericque Thompson','GA'],
                                                   ['Tabronx W Smith', 'GA']]
                                      },
             'Rare Breed Nation': {'State': 'GA',
                                    'Source': 'https://www.justice.gov/opa/pr/six-charged-connection-3-million-paycheck-protection-program-fraud-scheme',
                                    'Suspects': [['Rodericque Thompson', 'GA'],
                                                 ['Thomas D Wilson', 'GA']]
                                    },
             'Blue Pearl': {'State' : 'VA',
                            'Source' : 'https://www.hselaw.com/files/PPP/Statement_of_Facts_-_Jaafar_-_20-cr-185_-_20.08.25.pdf',
                            'Suspects': [['Tarik Jaafar', 'VA'], 
                                         ['Monika Magdalena Jaworska', 'VA']]
                            },
             'Global Capital Financing': {'State': 'VA',
                                          'Source': 'https://www.hselaw.com/files/PPP/Statement_of_Facts_-_Jaafar_-_20-cr-185_-_20.08.25.pdf',
                                          'Suspects': [['Tarik Jaafar', 'VA'], 
                                                       ['Monika Magdalena Jaworska', 'VA']]
                                          },
             'Washington Capital Group': {'State': 'VA',
                                           'Source': 'https://www.hselaw.com/files/PPP/Statement_of_Facts_-_Jaafar_-_20-cr-185_-_20.08.25.pdf',
                                           'Suspects': [['Tarik Jaafar', 'VA'], 
                                                        ['Monika Magdalena Jaworska', 'VA']]
                                            },
             'Visla Captial LLC': {'State': 'VA',
                               'Source': 'https://www.hselaw.com/files/PPP/Statement_of_Facts_-_Jaafar_-_20-cr-185_-_20.08.25.pdf',
                               'Suspects': [['Tarik Jaafar', 'VA'], 
                                            ['Monika Magdalena Jaworska', 'VA']]
                                  },
             'Maturino Enterprises': {'State': 'OK',
                                      'Source': 'https://www.justice.gov/usao-ndok/pr/two-men-charged-after-fraudulently-applying-paycheck-protection-program-loans',
                                      'Suspects': [['Rafael Maturino', 'OK'], 
                                                   ['Adam Winston James', 'OK']]
                                         },
             'Velocity Innovations': {'State': 'OK',
                                      'Source': 'https://www.justice.gov/usao-ndok/pr/two-men-charged-after-fraudulently-applying-paycheck-protection-program-loans',
                                      'Suspects': [['Rafael Maturino', 'OK'], 
                                                   ['Adam Winston James', 'OK']]
                                          },
             'AboveAll Business': {'State': 'OK',
                                      'Source': 'https://www.justice.gov/usao-ndok/pr/tulsa-man-sentenced-applying-paycheck-protection-program-loans-under-false-pretenses',
                                      'Suspects': [['Ibanga Etuk', 'OK'], 
                                                   ['Teosha Etuk', 'OK']]
                                      },
             'TMARK Enterprises Inc': {'State': 'OK',
                                     'Source': 'https://www.justice.gov/usao-ndok/pr/tulsa-man-sentenced-applying-paycheck-protection-program-loans-under-false-pretenses',
                                     'Suspects': [['Ibanga Etuk', 'OK'], 
                                                   ['Teosha Etuk', 'OK']]
                                     },
             'Pinnacle Industry Inc': {'State': 'OK',
                                    'Source': 'https://www.justice.gov/usao-ndok/pr/man-pleads-guilty-applying-paycheck-protection-program-loan-under-false-pretenses',
                                    'Suspects': ['Adewale Matthew Abel', 'OK']
                                    },
             'Maturino Enterprises': {'State': 'OK',
                                       'Source': 'https://www.justice.gov/usao-ndok/pr/broken-arrow-man-pleads-guilty-fraudulently-applying-paycheck-protection-program',
                                       'Suspects': ['Rafael Maturino', 'OK']
                                       },
             'RK Painting Company': {'State': 'MI',
                                     'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                                     'Suspects': [['Ricky Dixon', 'MI'],
                                                   ['Meghan Thomas', 'GA'],
                                                   ['Jesika Blakely', 'GA'],
                                                   ['Amanda Christian', 'SC'],
                                                   ['Dwan Ashong', 'FL']]
                                                   },
             'Advertising and Then Some Inc': {'State': 'MN',
                                               'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                                               'Suspects': [['Ricky Dixon', 'MI'],
                                                            ['Meghan Thomas', 'GA'],
                                                            ['Jesika Blakely', 'GA'],
                                                            ['Amanda Christian', 'SC'],
                                                            ['Dwan Ashong', 'FL']]
                                               },
             'Gaines Reservation and Travel': {'State': 'GA',
                                               'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                                               'Suspects': [['John Gaines', 'GA'],
                                                            ['Carla Jackson', 'GA'],
                                                            ['Andre Lee Gaines', 'GA']]
                                               },
            'Transportation Management Services': {'State': 'MN',
                                                    'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                                                    'Suspects': [['Charles Petty', 'GA'],
                                                                 ['Jerry Baptiste', 'GA'],
                                                                 ['Denesseria Slaton', 'GA'],
                                                                 ['Bern Benoit', 'CA'],
                                                                 ['Carla Jackson', 'GA']]
                                                    },
            'D Parker Holdings Inc': {'State': 'MI',
                                   'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                                   'Suspects': [['Derek Parker', 'MI']]
                                   },
            'Continuing Success Inc': {'State': 'SC',
                                        'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                                        'Suspects': [['David Belgrave II', 'SC']]
                                        },
            'Infinite Education Services Inc': {'State': 'GA',
                                                 'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                                                 'Suspects': [['Charles Hill IV', 'GA']]
                                                 },
            'ML Exotic Customs Inc': {'State': 'Il',
                                       'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                                       'Suspects': [['Ryan Whittley', 'IL']]
                                       },
            'Bellevie Corp': {'State': 'FL',
                              'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                              'Suspects': [['El Hadj Sall', 'FL']]
                              },
            'Mickies Auto and Tires LLC': {'State': 'SC',
                                           'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                                           'Suspects': [['Rick McDuffie', 'SC']]
                                          },
            'Bellator Phront Group Inc': {'State': 'GA',
                                           'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                                           'Suspects': [['Teldrin Foster', 'GA'],
                                                        ['Darrell Thomas', 'GA'],
                                                        ['Carla Jackson', 'GA']]
                                          },
            'All Star Room and Board Services of Michigan Inc': {'State': 'MI',
                                                                 'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                                                                 'Suspects': [['Charmaine Redding', 'MI']]
                                                                  },
            'Impact Creations LLC': {'State': 'OH',
                                      'Source': 'https://www.justice.gov/opa/pr/twenty-two-charged-connection-more-11-million-paycheck-protection-program-fraud-scheme',
                                      'Suspects': [['Khalil Gibran Green Sr', 'OH'],
                                                   ['Carla Jackson', 'GA']]
                                      },
            'Zen Solutions Inc': {'State': 'VA',
                                  'Source': 'https://www.justice.gov/opa/pr/northern-virginia-company-settles-false-claims-act-allegations-improper-paycheck-protection',
                                  'Suspects': [['No Individuals', 'VA']]
                                  },
            'Lee Operations LLC': {'State': 'GA',
                                   'Source': 'https://www.justice.gov/opa/pr/five-charged-connection-over-4-million-paycheck-protection-program-fraud-scheme',
                                   'Suspects': [['Andre Lee Gaines', 'GA'],
                                                 ['Carla Jackson', 'GA']]
                                   },
            '713 Construction LLC': {'State': 'TX',
                                     'Source': 'https://www.justice.gov/opa/pr/texas-man-pleads-guilty-federal-charges-fraudulently-obtaining-over-16-million-paycheck',
                                     'Suspects': [['Lee Price III', 'TX']]
                                     },
            'Price Enterprises Holdings LLC': {'State': 'TX',
                                               'Source': 'https://www.justice.gov/opa/pr/texas-man-pleads-guilty-federal-charges-fraudulently-obtaining-over-16-million-paycheck',
                                               'Suspects': [['Lee Price III', 'TX']]
                                               },
            'BSC Opportunistic Equity LP': {'State': 'NY',
                                            'Source': 'https://www.justice.gov/opa/pr/florida-man-pleads-guilty-federal-charges-fraudulently-obtaining-and-laundering-more-4',
                                            'Suspects': [['Gregory J Blotnick', 'FL']]
                                            },
            'Brattle Street GP LLC': {'State': 'NY',
                                      'Source': 'https://www.justice.gov/opa/pr/florida-man-pleads-guilty-federal-charges-fraudulently-obtaining-and-laundering-more-4',
                                      'Suspects': [['Gregory J Blotnick', 'FL']]
                                      },
            'Wild Stylz Entertainment LLC': {'State': 'GA',
                                             'Source': 'https://www.justice.gov/opa/pr/seven-charged-connection-21-million-money-laundering-scheme-involved-money-paycheck',
                                             'Suspects': [['Lauren Marcel Duhart', 'GA'], 
                                                          ['Joshua Bernard Smith', 'GA'],
                                                          ['Steve Ronald Lewis', 'GA'], 
                                                          ['Christopher J Agard', 'GA'], 
                                                          ['Henry Duffield', 'SC'], 
                                                          ['Jeremy Brandon Latourneau', 'SC'],
                                                          ['Derick Keane', 'SC']]
                                             },
          'Top Notch Tile': {'State': 'MA',
                             'Source': 'https://www.justice.gov/usao-ri/press-release/file/1318951/download',
                             'Suspects': [['Michael Moller', 'MA']]
                              },
          'Randal Moller dba TNT Tile': {'State': 'MA',
                       'Source': 'https://www.justice.gov/usao-ri/press-release/file/1318951/download',
                       'Suspects': [['Michael Moller', 'MA']]
                              },
          'A Top Notch Remodel':{'State': 'MA',
                                 'Source': 'https://www.justice.gov/usao-ri/press-release/file/1318951/download',
                                 'Suspects': [['Michael Moller', 'MA']]
                                },
          'Sextant Marine Consulting LLC': {'State': 'FL',
                                             'Source': 'https://www.justice.gov/opa/pr/covid-19-task-force-nets-florida-duct-cleaning-company-settles-false-claims-act-allegations',
                                             'Suspects': [['No Individuals', 'FL']]
                                            },
          'Palmers Transportation Inc': {'State': 'TN',
                                         'Source': 'https://www.justice.gov/usao-mdtn/pr/smyrna-man-charged-paycheck-protection-program-fraud',
                                         'Suspects': [['Shawn Palmer', 'TN']]
                                         },
          'Kurbanov Communications LLC': {'State': 'MI',
                                          'Source': 'https://www.hselaw.com/files/PPP/4-13-22/Defendants_Sentencing_Memo_D_Kurbanov_-_J_Mason_et_al_-_21-cr-69_-_2022.02.24.pdf',
                                          'Suspects': [['David Kurbanov', 'MI']]},
          'In a Minute Entertainment LLC': {'State': 'MI',
                                            'Source': 'https://www.hselaw.com/files/PPP/4-13-22/Defendants_Sentencing_Memo_D_Kurbanov_-_J_Mason_et_al_-_21-cr-69_-_2022.02.24.pdf',
                                            'Suspects': [['David Kurbanov', 'MI']]
                                            },
          'Silver Cup Services Group LLC': {'State': 'NJ',
                                            'Source': 'https://www.justice.gov/usao-nj/pr/camden-county-man-admits-fraudulently-obtaining-paycheck-protection-program-loan',
                                            'Suspects': [['Cornell McCoy', 'NJ']]
                                            },
          'Blue Flight Logistics LLC': {'State': 'VA',
                                        'Source': 'https://www.justice.gov/usao-ndia/pr/virginia-man-pleads-guilty-paycheck-protection-program-money-laundering-scheme',
                                        'Suspects': [['Benjamin Sakyi', 'VA'],
                                                     ['Donald Franklin Trosin', 'MN']]
                                        },
          'NKB Enterprise LLC': {'State': 'VA',
                                 'Source': 'https://www.justice.gov/usao-ndia/pr/virginia-man-pleads-guilty-paycheck-protection-program-money-laundering-scheme',
                                 'Suspects': [['Benjamin Sakyi', 'VA'],
                                              ['Donald Franklin Trosin', 'MN']]
                                },
          'Extract LLC': {'State': 'OH',
                          'Source': 'https://www.justice.gov/usao-sdoh/pr/dayton-business-owner-charged-covid-relief-fraud',
                          'Suspects': [['Nadine Consuelo Jackson', 'OH']]
                          },
          'True Cut Construction LLC': {'State': 'OH',
                                        'Source': 'https://www.justice.gov/opa/pr/minnesota-man-charged-covid-relief-fraud-and-money-laundering',
                                        'Suspects': [['Kyle William Brenizer', 'MN']]
                                        },
          'Eric R Shibley MD PLLC': {'State': 'WA',
                                     'Source': 'https://www.hselaw.com/files/11-12/Govt_Trial_Brief_-_Shibley_-_20-cr-174_-_2021.11.01.pdf',
                                     'Suspects': [['Eric Shibley', 'WA']]
                                     },
          'SS1 LLC': {'State': 'WA',
                      'Source': 'https://www.hselaw.com/files/11-12/Govt_Trial_Brief_-_Shibley_-_20-cr-174_-_2021.11.01.pdf',
                      'Suspects': [['Eric Shibley', 'WA']]
                      },
          'ES1 LLC': {'State': 'WA',
                      'Source': 'https://www.hselaw.com/files/11-12/Govt_Trial_Brief_-_Shibley_-_20-cr-174_-_2021.11.01.pdf',
                      'Suspects': [['Eric Shibley', 'WA']]
                      },
          'The A Team Holdings LLC': {'State': 'WA',
                                      'Source': 'https://www.hselaw.com/files/11-12/Govt_Trial_Brief_-_Shibley_-_20-cr-174_-_2021.11.01.pdf',
                                      'Suspects': [['Eric Shibley', 'WA']]
                                      },
          'Dituri Construction LLC': {'State': 'WA',
                                      'Source': 'https://www.hselaw.com/files/11-12/Govt_Trial_Brief_-_Shibley_-_20-cr-174_-_2021.11.01.pdf',
                                      'Suspects': [['Eric Shibley', 'WA']]
                                      },
          'SFC LLC': {'State': 'WA',
                      'Source': 'https://www.hselaw.com/files/11-12/Govt_Trial_Brief_-_Shibley_-_20-cr-174_-_2021.11.01.pdf',
                      'Suspects': [['Eric Shibley', 'WA']]
                      },
          'Seattles Finest Cannabis LLC': {'State': 'WA',
                                           'Source': 'https://www.hselaw.com/files/11-12/Govt_Trial_Brief_-_Shibley_-_20-cr-174_-_2021.11.01.pdf',
                                           'Suspects': [['Eric Shibley', 'WA']]
                                           },
          'Speedez Cleaners LLC': {'State': 'WY',
                                   'Source': 'https://www.justice.gov/usao-nj/press-release/file/1377291/download',
                                   'Suspects': [['John Jhong ', 'NJ']]
                                   },
          'Central Staffing LLC': {'State': 'WY',
                                   'Source': 'https://www.justice.gov/usao-nj/press-release/file/1377291/download',
                                   'Suspects': [['John Jhong ', 'NJ']]
                                   },
          'Lolas Level': {'State': 'TX',
                          'Source': 'https://www.justice.gov/usao-sdtx/pr/texas-woman-charged-fraudulently-obtaining-nearly-2m-covid-relief-funds',
                          'Suspects': [['Lola Shalewa Barbara Kasali', 'TX']]
                          },         
          'Texas Barbecue LLC': {'State': 'TX',
                             'Source': 'https://www.justice.gov/usao-sdtx/pr/another-houston-man-charged-covid-relief-fraud',
                             'Suspects': [['Joshua Thomas Argires', 'TX']]
                             },
          'Houston Landscaping LLC': {'State': 'TX',
                                      'Source': 'https://www.justice.gov/usao-sdtx/pr/another-houston-man-charged-covid-relief-fraud',
                                      'Suspects': [['Joshua Thomas Argires', 'TX']]
                                      },
         'ENI Marketing Inc': {'State': 'TX',
                               'Source': 'https://www.justice.gov/criminal-fraud/file/1315221/download',
                               'Suspects': [['Jase DePaul Gautreaux', 'TX']]
                               },
         'ENI Operation Co': {'State': 'TX',
                              'Source': 'https://www.justice.gov/criminal-fraud/file/1315221/download',
                              'Suspects': [['Jase DePaul Gautreaux', 'TX']]
                              },
         'Wingate Funeral Service Group': {'State': 'TX',
                                           'Source': 'https://www.justice.gov/criminal-fraud/file/1315221/download',
                                           'Suspects': [['Jase DePaul Gautreaux', 'TX']]
                                          },
        'Showtime Trucking LLC': {'State': 'NJ',
                                  'Source': 'https://www.justice.gov/usao-nj/press-release/file/1479526/download',
                                  'Suspects': [['Jean E Rabbitt', 'TX'],
                                              ['Kevin Aguilar', 'TX']]
                                              }, 
        'Showtime Trucking XPO Inc': {'State': 'NJ',
                                      'Source': 'https://www.justice.gov/usao-nj/press-release/file/1479526/download',
                                      'Suspects': [['Jean E Rabbitt', 'TX'],
                                                  ['Kevin Aguilar', 'TX']]
                                                  },     
        'TTS Terminal Corporation': {'State': 'NJ',
                                    'Source': 'https://www.justice.gov/usao-nj/press-release/file/1479526/download',
                                    'Suspects': [['Jean E Rabbitt', 'TX'],
                                                ['Kevin Aguilar', 'TX']]
                                                },    
        'American Consolidated Freightways Corporation': {'State': 'NJ',
                                                          'Source': 'https://www.justice.gov/usao-nj/press-release/file/1479526/download',
                                                          'Suspects': [['Jean E Rabbitt', 'TX'],
                                                                      ['Kevin Aguilar', 'TX']]
                                                                      },    
        'Kloudgaze Inc': {'State': 'MN',
                    'Source': 'https://www.justice.gov/usao-mn/pr/maple-grove-man-charged-defrauding-us-small-business-administration-s-paycheck-protection',
                    'Suspects': [['Aditya Raj Sharma', 'MN']]
                    },
        'Therapetic Solutions;': {'State': 'DC',
                    'Source': 'https://www.hselaw.com/files/PPP/PPP_5/Indictment_-_Gaughan_-_21-cr-390_-_2021.06.07.pdf',
                    'Suspects': [['Kenneth Patrick Gaughan', 'DC']]
                    },
       'Therapy Dog International': {'State': 'DC',
                    'Source': 'https://www.hselaw.com/files/PPP/PPP_5/Indictment_-_Gaughan_-_21-cr-390_-_2021.06.07.pdf',
                    'Suspects': [['Kenneth Patrick Gaughan', 'DC']]
                    },
        'ESA Registry International': {'State': 'DC',
                    'Source': 'https://www.hselaw.com/files/PPP/PPP_5/Indictment_-_Gaughan_-_21-cr-390_-_2021.06.07.pdf',
                    'Suspects': [['Kenneth Patrick Gaughan', 'DC']]
                    },
        'Certapet Inc': {'State': 'DC',
                    'Source': 'https://www.hselaw.com/files/PPP/PPP_5/Indictment_-_Gaughan_-_21-cr-390_-_2021.06.07.pdf',
                    'Suspects': [['Kenneth Patrick Gaughan', 'DC']]
                    },
        'Therapy Dog Inc': {'State': 'DC',
                    'Source': 'https://www.hselaw.com/files/PPP/PPP_5/Indictment_-_Gaughan_-_21-cr-390_-_2021.06.07.pdf',
                    'Suspects': [['Kenneth Patrick Gaughan', 'DC']]
                    },
        'Official Service Dogs': {'State': 'DC',
                    'Source': 'https://www.hselaw.com/files/PPP/PPP_5/Indictment_-_Gaughan_-_21-cr-390_-_2021.06.07.pdf',
                    'Suspects': [['Kenneth Patrick Gaughan', 'DC']]
                    },
        '1Stellar Health LLC': {'State': 'CA',
                    'Source': 'https://www.justice.gov/opa/pr/man-convicted-27-million-ppp-fraud-scheme',
                    'Suspects': [['Robert Benlevi', 'CA']]
                    },
        'Bestways2 Health LLC': {'State': 'CA',
                    'Source': 'https://www.justice.gov/opa/pr/man-convicted-27-million-ppp-fraud-scheme',
                    'Suspects': [['Robert Benlevi', 'CA']]
                    },
        'JoyousHealth4U LLC': {'State': 'CA',
                    'Source': 'https://www.justice.gov/opa/pr/man-convicted-27-million-ppp-fraud-scheme',
                    'Suspects': [['Robert Benlevi', 'CA']]
                    },
        'La Salette Angel Home Care': {'State': 'MA',
                    'Source': 'https://www.pagepate.com/updated-list-of-ppp-and-eidl-loan-fraud-criminal-cases/',
                    'Suspects': [['Adley Bernadin', 'MA']]
                    },
        'Williams Consulting Group LLC': {'State': 'FL',
                    'Source': 'https://www.justice.gov/usao-sdfl/pr/local-basketball-coach-charged-federal-court-defrauding-covid-19-relief-program-out',
                    'Suspects': [['Terrence Deshun Williams', 'FL']]
                    },
        'Papillon Holdings Incorporated': {'State': 'VA',
                    'Source': 'https://apnews.com/article/business-pandemics-virginia-coronavirus-pandemic-2e7c551e17ad6880ea30d59bff329200',
                    'Suspects': [['Didier Kindambu', 'VA']]
                    },
        'Papillon Air Inc': {'State': 'VA',
                    'Source': 'https://apnews.com/article/business-pandemics-virginia-coronavirus-pandemic-2e7c551e17ad6880ea30d59bff329200',
                    'Suspects': [['Didier Kindambu', 'VA']]
                    },
        'SARC Inc': {'State': 'MA',
                    'Source': 'https://www.hselaw.com/files/PPP/Oct_9_2021/Complaint_-_J_Cohen_-_21-mj-2635_-_2021.09.27.pdf',
                    'Suspects': [['James Joseph Cohen', 'MA']]},
        'Ceromaze Inc': {'State': 'MA',
                    'Source': 'https://www.hselaw.com/files/PPP/Oct_9_2021/Complaint_-_J_Cohen_-_21-mj-2635_-_2021.09.27.pdf',
                    'Suspects': [['James Joseph Cohen', 'MA']]},
        'Aqeel Amir DBA Irs Services': {'State': 'TX',
                                        'Source': 'https://projects.propublica.org/coronavirus/bailouts/loans/aqeel-amir-dba-irs-services-4057298006',
                                         'Suspects': [['Amir Aqeel', 'TX'],
                                                      ['Siddiq Azeemuddin', 'IL'],
                                                      ['Rifat Bajwa', 'TX'],
                                                      ['Pardeep Basra', 'TX'],
                                                      ['Mayer Misa', 'TX'],
                                                      ['Mauricio Navia', 'TX'],
                                                      ['Richard Reuth', 'TX']]
                                        },
        'AF Logistics LLC': {'State': 'TX',
                             'Source': 'https://www.justice.gov/criminal-fraud/file/1354381/download',
                             'Suspects': [['Amir Aqeel', 'TX'],
                                          ['Siddiq Azeemuddin', 'IL'],
                                          ['Rifat Bajwa', 'TX'],
                                          ['Pardeep Basra', 'TX'],
                                          ['Mayer Misa', 'TX'],
                                          ['Mauricio Navia', 'TX'],
                                          ['Richard Reuth', 'TX']]
                            },
        'Champion Automower Inc': {'State': 'TX',
                                   'Source': 'https://www.justice.gov/criminal-fraud/file/1354381/download',
                                   'Suspects': [['Amir Aqeel', 'TX'],
                                                ['Siddiq Azeemuddin', 'IL'],
                                                ['Rifat Bajwa', 'TX'],
                                                ['Pardeep Basra', 'TX'],
                                                ['Mayer Misa', 'TX'],
                                                ['Mauricio Navia', 'TX'],
                                                ['Richard Reuth', 'TX']]
                                  },
        'Grandeur Construction LLC': {'State': 'TX',
                                      'Source': 'https://www.justice.gov/criminal-fraud/file/1354381/download',
                                      'Suspects': [['Amir Aqeel', 'TX'],
                                                   ['Siddiq Azeemuddin', 'IL'],
                                                   ['Rifat Bajwa', 'TX'],
                                                   ['Pardeep Basra', 'TX'],
                                                   ['Mayer Misa', 'TX'],
                                                   ['Mauricio Navia', 'TX'],
                                                   ['Richard Reuth', 'TX']]
                                    },
        'Houston Electronic Group LLC': {'State': 'TX',
                                         'Source': 'https://www.justice.gov/criminal-fraud/file/1354381/download',
                                         'Suspects': [['Amir Aqeel', 'TX'],
                                                      ['Siddiq Azeemuddin', 'IL'],
                                                      ['Rifat Bajwa', 'TX'],
                                                      ['Pardeep Basra', 'TX'],
                                                      ['Mayer Misa', 'TX'],
                                                      ['Mauricio Navia', 'TX'],
                                                      ['Richard Reuth', 'TX']]
                                        },  
        'Kata Services LLC': {'State': 'TX',
                              'Source': 'https://www.justice.gov/criminal-fraud/file/1354381/download',
                              'Suspects': [['Amir Aqeel', 'TX'],
                                           ['Siddiq Azeemuddin', 'IL'],
                                           ['Rifat Bajwa', 'TX'],
                                           ['Pardeep Basra', 'TX'],
                                           ['Mayer Misa', 'TX'],
                                           ['Mauricio Navia', 'TX'],
                                           ['Richard Reuth', 'TX']]
                              },  
        'Sanad Inc': {'State': 'TX',
                      'Source': 'https://www.justice.gov/criminal-fraud/file/1354381/download',
                      'Suspects': [['Amir Aqeel', 'TX'],
                                   ['Siddiq Azeemuddin', 'IL'],
                                   ['Rifat Bajwa', 'TX'],
                                   ['Pardeep Basra', 'TX'],
                                   ['Mayer Misa', 'TX'],
                                   ['Mauricio Navia', 'TX'],
                                   ['Richard Reuth', 'TX']]
                      }, 
        'Popeshenouda': {'State': 'TX',
                         'Source': 'https://www.justice.gov/criminal-fraud/file/1354381/download',
                         'Suspects': [['Amir Aqeel', 'TX'],
                                      ['Siddiq Azeemuddin', 'IL'],
                                      ['Rifat Bajwa', 'TX'],
                                      ['Pardeep Basra', 'TX'],
                                      ['Mayer Misa', 'TX'],
                                      ['Mauricio Navia', 'TX'],
                                      ['Richard Reuth', 'TX']]
                      },
        'US Auto Exchange Inc': {'State': 'TX',
                                 'Source': 'https://www.justice.gov/criminal-fraud/file/1354381/download',
                                 'Suspects': [['Amir Aqeel', 'TX'],
                                              ['Siddiq Azeemuddin', 'IL'],
                                              ['Rifat Bajwa', 'TX'],
                                              ['Pardeep Basra', 'TX'],
                                              ['Mayer Misa', 'TX'],
                                              ['Mauricio Navia', 'TX'],
                                              ['Richard Reuth', 'TX']]
                      },
        'Youkeva LLC': {'State': 'TX',
                                 'Source': 'https://www.justice.gov/criminal-fraud/file/1354381/download',
                                 'Suspects': [['Amir Aqeel', 'TX'],
                                              ['Siddiq Azeemuddin', 'IL'],
                                              ['Rifat Bajwa', 'TX'],
                                              ['Pardeep Basra', 'TX'],
                                              ['Mayer Misa', 'TX'],
                                              ['Mauricio Navia', 'TX'],
                                              ['Richard Reuth', 'TX']]
                      },
        'The Little Piglet Soap Company LLC': {'State': 'AK',
                    'Source': 'https://www.justice.gov/usao-edar/pr/little-rock-woman-charged-covid-relief-fraud',
                    'Suspects': [['Ganell Tubbs', 'AK']]},
        'Suga Girl Customs LLC': {'State': 'AK',
                    'Source': 'https://www.justice.gov/usao-edar/pr/little-rock-woman-charged-covid-relief-fraud',
                    'Suspects': [['Ganell Tubbs', 'AK']]},
        'Ruth Properties MGM': {'State': 'MO',
                    'Source': 'https://www.joplinglobe.com/coronavirus/missouri-man-pleads-guilty-to-pandemic-business-loan-fraud/article_cc544ed0-ae92-11eb-b7bb-63d2599f94e4.html',
                    'Suspects': [['Brian Ruth Sr', 'MO']]},
        'Pink Lady Line': {'State': 'TX',
                    'Source': 'https://www.justice.gov/usao-sdtx/pr/local-woman-charged-fraudulently-receiving-millions-under-cares-act',
                    'Suspects': [['LaDonna Wiggins', 'TX']]},
        'Guardian of Humanity Inc': {'State': 'AZ',
                    'Source': 'https://www.justice.gov/usao-wdny/pr/north-chili-man-charged-covid-relief-fraud-scheming-get-paycheck-protection-program',
                    'Suspects': [['Michael Rech', 'NY']]},
        'Skilled Trade Investments LP': {'State': 'TX',
                    'Source': 'https://www.justice.gov/opa/pr/texas-man-arrested-covid-19-relief-scheme',
                    'Suspects': [['Scott Jackson Davis', 'TX']]},
        'Skilled Trade Staffing LLC': {'State': 'TX',
                    'Source': 'https://www.justice.gov/opa/pr/texas-man-arrested-covid-19-relief-scheme',
                    'Suspects': [['Scott Jackson Davis', 'TX']]},
        'Skilled Trade Investments Gp LLC': {'State': 'TX',
                    'Source': 'https://www.justice.gov/opa/pr/texas-man-arrested-covid-19-relief-scheme',
                    'Suspects': [['Scott Jackson Davis', 'TX']]},
        
} 

In [ ]:
ppp_fraud_cases_df = pd.DataFrame.from_dict(ppp_fraud_cases).T.reset_index()
ppp_fraud_cases_df.rename(columns = {'index': 'Company'}, inplace=True)
ppp_fraud_cases_df.shape

In [ ]:
ppp_fraud_cases_df['Company'] = format_name(df=ppp_fraud_cases_df, col='Company')

In [ ]:
ppp_fraud_cases_df.info()

In [ ]:
ppp_fraud_cases_df.head()

### Check Fraud Company Names

In [ ]:
fraud_company_names = ppp_fraud_cases_df['Company'].unique().tolist()
all_company_names = df['BorrowerName'].unique().tolist()

In [ ]:
[x for x in fraud_company_names if x not in all_company_names]

In [ ]:
# # query to check names 
# df[df['BorrowerName'].str.contains('us auto exchange', flags=re.IGNORECASE)]

## Output Data to CSVs

### PPP Over 150k and Data Dictionary

In [ ]:
# df.info()

In [ ]:
df.to_csv('ppp_over_150k.csv', index=None)

In [ ]:
ppp_dict_v2.to_csv('ppp_data_dict.csv', index=None)

In [ ]:
ppp_fraud_cases_df.to_csv('ppp_fraud_cases.csv', index=None)